# B.6. Parallel programming models (OpenMP)

OpenMP is an application programming interface that supports multi-platform shared-memory multiprocessing programming in C, C++, and Fortran, on many platforms, instruction-set architectures and operating systems, including Solaris, AIX, FreeBSD, HP-UX, Linux, macOS, and Windows.

## 0. Introduction to OpenMP
(Based on https://github.com/josemonsalve2/openmp_tutorial/)

The objective of this lab is to get familiar with the basic concepts behind OpenMP. Some of these concepts are shared with other programming models, and are important to understand how systems are programmed in parallel. These concepts are introduced directly using OpenMP syntax. It is not expected for the reader to know OpenMP, but they should be familiar with C-like syntax.

This tutorial is expected to run in a unix-like environment.

Table of content:
- First parallel program
- Thinking in parallel
- Exercise 1
- Memory: Shared, private, distributed
- Atomic operations
- Private vs Firstprivate
- Reductions
- Lastprivate
- Exercise 2
- OpenMP Syntax
- Function outlining, implementation and runtime




## 1. First parallel program
Let's create our first threaded program

```
#include <omp.h>

int main() {
    #pragma omp parallel num_threads(10)
    {
        printf("Hello from thread %d\n",omp_get_thread_num());
    }
    return 0;
}
```
Let's compile it!

In [ ]:
!srun gcc -fopenmp Code/omp_hello_world.c -o Code/omp_hello_world.o

Now it is time to run it.

In [ ]:
!srun ./Code/omp_hello_world.o 

You should notice that (likely) the output of the above program is not in a given order. This is because all the threads are running concurrently, and, if the number of hardware threads is larger than 1, a set of them may be running in parallel.

Open the file omp_hello_world.c and play with different number of threads by changing the value inside the clause num_threads(), and re-running the above two commands.

You can check this code in [hello_world.c](Code/omp_hello_world.c).

## Thinking in parallel
If you're experienced in sequential programming, you're likely familiar with writing code for a single thread. When thinking sequentially, the developer must primarly think of the instructions that are executed. Parallel programming adds an additional complexity to the development program. When writting programs, a developer must think of the following aspects of the code:

Workers creation: How to create workers and how many to create
Work assignment: How to assign work to different workers
Workers/resources communication and coordination: How workers communicate and synchronize in order to coordinate their work.
Note: Different programming models exist that balance these three tasks. Here I am referring to the most popular programming models. Allow me to be simplistic here.
The fork-join model - An example.
OpenMP is mainly known for its Fork-Join model. Programs execute with an initial sequential thread until a directive (code anotation) is reached that initiates parallel threads (workers). Different directives are used to assign work to these threads. At the end of the parallel region, threads synchronize, forming again a single thread.

For the above example. worker creation is achieved through the #pragma omp parallel num_threads(10) directive. It allows to create 10 threads. Each worker is associated with an identifier, obtained through the omp_get_thread_num() function call. Work assignment is achieved through the code in the region enclosed by the brackets {...}. All threads are executing the same set of instructions, in this case only printf("hello from thread %d\n",...);. Notice that having a different identifier allows for each thread to access different data, or follow different paths. Finally Worker synchronization and coordination is relatively simple in this example, since there's not much communication between them. However, at the end of the parallel region, all workers must wait for each other to finish before continuing with the sequential code.

## Exercise 1
Can you modify the code above to make the odd and even threads print something different?

Go to exercise1.c and use the code region below to build and execute your code

In [ ]:
!srun gcc -fopenmp Exercises/omp_exercise_1.c -o Exercises/omp_exercise_1.o && Exercises/./omp_exercise_1.o

See a possible solution to Exercise 1.

In [ ]:
!srun gcc -fopenmp Solutions/omp_exercise_1.c -o Solutions/omp_exercise_1.o && Solutions/./omp_exercise_1.o

In [ ]:
!srun gcc -fopenmp Code/omp_memory.c -o Code/omp_memory.o

!srun ./Code/omp_memory.o 

Private memory allows for variables to be visible only by the current worker. Shared memory, allows variables to be visible from and modified by multiple workers (read and write). However, it is important to be careful. As previously mentioned, memory organization is complex, and requires additional coordination.

Take for example the following program:

```
int main() {
    int i = 0;

    #pragma omp parallel shared(i) num_threads(10000)
    {
        i++;
    }
    
    printf("i = %d\n",i);
    return 0;
}
```

In [ ]:
!srun gcc -fopenmp Code/omp_datarace.c -o Code/omp_datarace.o

In [ ]:
!srun ./Code/omp_datarace.o 

If 10000 workers are created, and each is adding 1 to the value of i, the "expected" value would be 10000. If you're running this in a machine that has multiple threads, I expect you to see a number less than 10000. Moreover, multiple executions may lead to different results. This is what is known as a data race. It occurs because reading i, incrementing, and writting to i are three different instructions. Therefore, it is possible for two threads to write the same value of i, increment it and obtain the same value twice.

For this reason, shared memory requires additional coordination between workers, such that reads and writes to the same region are perceived in the expected order.

Notice that this is part of thinking about Workers/resource communication and coordination. We are coordinating memory acesses and operations to variables, as they are shared across different workers.

You can modify the above code going to omp_datarace.c

Note: Data races are a difficult aspect of parallel programming. Multiple executions may lead to different results, but among the different results that can be obtained, it is still possible to obtain the "expected" result. Some data races are more difficult to debug than others, because they have a higher chance of choosing the "expected" result. Imagine that 1 out of a million executions of your code in a given hardware shows a datarace.

## Atomic operations

Atomic operations allows different threads to perform different operations to the same memory location in a single instruction (or as if they were executed in a single instruction). In the example above, atomic operations would allow for read, increment and write to happen in a single instruction (or "atomically"). Thus, atomic operations can solve the datarace issue of the previous example.

```
int main() {
    int i = 0;

    #pragma omp parallel shared(i) num_threads(10000)
    {
        #pragma omp atomic
        i++;
    }
    
    printf("i = %d\n",i);
    return 0;
}
```

## Memory: Shared vs Private
When programming we often imagine memory as a single "monolithic" element. However, this is not the reality. Current architectures feature a complex memory organization that includes registers, caches, multiple DRAM banks, and even devices with a different memory space. When programming in parallel, such complex structures become more important for correctness and performance.

Multithreading programming often features shared memory, meaning that all threads have access to the same varibles, located in the same memory address (I expect the reader to be familiar with pointers). However, it is also possible for each thread to have private memory, even if the name of variables are the same.

Take for example the next program:

```
int main() {
    int i;
    double share;
    int Array[10];

    printf("Address of i prior to the parallel region is: %lx\n",(unsigned long)&i);
    printf("Address of shared prior to the parallel region is: %lx\n",(unsigned long)&share);
    printf("Address of Array prior to the parallel region is: %lx\n",(unsigned long)Array);

    #pragma omp parallel private(i, Array) shared(share)
    {
        printf("Address of i as seen by thread %d: %lx\n", omp_get_thread_num(), (unsigned long)&i);
        printf("Address of Array as seen by thread %d: %lx\n", omp_get_thread_num(), (unsigned long)Array);
        printf("Address of share as seen by thread %d: %lx\n", omp_get_thread_num(), (unsigned long)&share);
    }
}
```

In [ ]:
!srun gcc -fopenmp Code/omp_atomic.c -o Code/omp_atomic.o

In [ ]:
!srun ./Code/omp_atomic.o 

You can play with the above code going to omp_atomic.c

## Private vs Firstprivate
Privatization of variables means that a single variable name will have different memory locations. Privatization does not guarantee that the new address has the same value of the original address (i.e. the address before the parallel region).

Take for example the following code:

```
int main() {
    int i = 999;

    #pragma omp parallel private(i) num_threads(10)
    {
        print("Thread %d sees %d", omp_get_thread_num(), i);
    }

    return 0;
}
```

In [ ]:
!srun gcc -fopenmp Code/omp_private.c -o Code/omp_private.o

In [ ]:
!srun ./Code/omp_private.o 

You can play with the above code going to omp_private.c

Firstprivate allows for each new address location to be initialized with the value prior to the parallel region. Take for example the following code.

```
int main() {
    int i[3] = {999,888,666};

    printf("i is [%d,%d,%d] before parallel region\n",i[0],i[1],i[2]);

    #pragma omp parallel firstprivate(i) num_threads(10)
    {
        printf("Thread %d sees [%d,%d,%d] on memory %lx\n", omp_get_thread_num(), i[0],i[1],i[2], (unsigned long)i);
    }

    return 0;
}
```


In [ ]:
!srun gcc -fopenmp Code/omp_firstprivate.c -o Code/omp_firstprivate.o

In [ ]:
!srun ./Code/omp_firstprivate.o 

You can play with the above code going to omp_firstprivate.c

Notice how each location for i is different on each thread, yet, all have the same expected value. This also means that memory copies need to be performed in order to achieve this behavior. Hence, if your array is large, it may incur in additional overhead.

There are still a lot of good reasons to privatize variables.

## Reductions
So far we have discussed what happens to variables when going from a sequential region to a parallel region. However, what happens to these multiple private memory locations when a parallel region is over? In principle, private variables are discarded (freed) at the end of a parallel region, hence, if their value are important, it is necessary to store them into a shared location that lives after the parallel region. Yet, it is often desireable to update the original memory location of the variable.

But wait, all of these variables may have different values. How do I decide what's the final value to be used after the parallel region? Reductions are collective operations that aggregate the different values into a single value, by applying an operation. Ideally, this operation should be commutative, otherwise, how do I decide the order in which they are applied?

Note: Surprisingly enough OpenMP used to support the minus (-) operation for reductions. It wasn't until version 5.2 that they removed support for this operation. If thread 1 and thread 2 are reducing A, should it be At1 - At2 or At2 - At1?
Let's take for example the following code:

```
int main() {
    int i = 99;

    printf("Value if i prior to parallel region is %d\n",i);

    // Private values are not transferred back
    #pragma omp parallel private(i)
    {
        i=1000;
    }
    printf("Value if i after parallel region with private(i) is %d\n",i);

    i = 0;
    // Reductions for addition.
    #pragma omp parallel reduction(+:i) num_threads(10)
    {
        i=1;
    }
    printf("Value if i after parallel region with reduction(+:i) is %d\n",i);

    // Reductions for max.
    #pragma omp parallel reduction(max:i) num_threads(20)
    {
        i=omp_get_thread_num();
    }
    printf("Value if i after parallel region with reduction(max:i) is %d\n",i);

    return 0;
}
```


In [ ]:
!srun gcc -fopenmp Code/omp_reductions.c -o Code/omp_reductions.o

In [ ]:
!srun ./Code/omp_reductions.o 

You can play with the above code going to omp_reductions.c. Other reduction operations are:
- Multiplication (*)
- Minimun (min)
- Bitwise AND (&)
- Bitwise OR (|)
- Bitwise XOR (^)
- Logic AND (&&)
- Logic OR (&&)tremendo 

## Lastprivate
Finally, there is lastprivate. Later on we will discuss more about loops and how to distribute them across workers. However, lastprivate allows for the value to be the last value in the iteration space. This means, if we have 10 iterations in a for loop, the value for i = 9 will be copied over.

This code shows this behavior. Let us ignore for now the for construct. We will go back to it later on.

```
int Array[10];
    int i, b;

    for (i = 0; i < 10; i++) {
        Array[i] = i;
    }

    #pragma omp parallel for lastprivate(b)
    for (i = 0; i < 10; i++)
    {
        b = Array[i];
    }
    printf("b is %d after the parallel region\n", b);

    return 0;
```

In [ ]:
!srun gcc -fopenmp Code/omp_lastprivate.c -o Code/omp_lastprivate.o

In [ ]:
!srun ./Code/omp_lastprivate.o 

You can play with this code going to omp_lastprivate.c.

### Exercise 2
Create a program that:

Initializes an array of 100 elements to random numbers by assigning a thread per element of the array.
- Finds the max value of the array.
- Finds the min value of the array.
- Finds the average value of the array.

Go to exercise2.c and use the code region below to build and execute your code

In [ ]:
!srun gcc -fopenmp Exercises/omp_exercise_2.c -o Exercises/omp_exercise_2.o && Exercises/./omp_exercise_2.o

See a possible solution to Exercise 2.

In [ ]:
!srun gcc -fopenmp Solutions/omp_exercise_2.c -o Solutions/omp_exercise_2.o && Solutions/./omp_exercise_2.o